In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [26]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [31]:
app = dash.Dash(__name__)

# ------------------------------------------------------------------------------
# Import and clean data (importing csv into pandas)
df = pd.read_csv("intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:5])

bee_killers = ["Disease", "Other", "Pesticides", "Pests_excl_Varroa", "Unknown", "Varroa_mites"]



app.layout = html.Div([
    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),
    
    html.Div([
        dcc.Dropdown(id="slct_impact",
                     options=[{"label": x, "value":x} for x in bee_killers],
                     value="Pesticides",
                     multi=False,
                     style={'width': "50%"}
                     ),
        dcc.Graph(id='my_line_graph', figure={})
    ], style={'display': 'inline-block', 'width': '49%'}),
    
    html.Div([
        dcc.Dropdown(id="slct_year",
                     options=[
                         {"label": "2015", "value": 2015},
                         {"label": "2016", "value": 2016},
                         {"label": "2017", "value": 2017},
                         {"label": "2018", "value": 2018}],
                     multi=False,
                     value=2015,
                     style={'width': "45%"}
                     ),
        dcc.Graph(id='my_bar_graph', figure={})
    ], style={'display': 'inline-block', 'width': '49%'}),
    
    html.Div([
        html.Button("Download HTML", id="btn-download"),
        dcc.Download(id="download")
    ])
])

@app.callback(
    [Output(component_id='my_line_graph', component_property='figure'),
     Output(component_id='my_bar_graph', component_property='figure')],
    [Input(component_id='slct_impact', component_property='value'),
     Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd1, option_slctd2):
    dff1 = df.copy()
    dff1 = dff1[dff1["Affected by"] == option_slctd1]
    dff1 = dff1[(dff1["State"] == "Idaho") | (dff1["State"] == "New York") | (dff1["State"] == "New Mexico")]
    
    fig1 = px.line(
        data_frame=dff1,
        x='Year',
        y='Pct of Colonies Impacted',
        color='State',
        template='plotly_dark'
    )
    
    dff2 = df.copy()
    dff2 = dff2[dff2["Year"] == option_slctd2]
    dff2 = dff2[dff2["Affected by"] == "Varroa_mites"]
    
    fig2 = px.bar(
        data_frame=dff2,
        x='State',
        y='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )
    
    return fig1, fig2

@app.callback(
    Output("download", "data"),
    [Input("btn-download", "n_clicks")],
    prevent_initial_call=True
)
def download_dashboard(n_clicks):
    html_string = app.to_html()
    return dict(content=html_string, filename="dashboard.html")
if __name__ == '__main__':
    app.run_server(debug=True)

     State  ANSI Affected by  Year state_code  Pct of Colonies Impacted
0  Alabama     1     Disease  2015         AL                      0.05
1  Alabama     1     Disease  2016         AL                      1.20
2  Alabama     1     Disease  2017         AL                      2.25
3  Alabama     1     Disease  2018         AL                      1.30
4  Alabama     1     Disease  2019         AL                      1.80
